<a href="https://colab.research.google.com/github/Buraporn-Subsomboon/Buraporn_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [ ]:
1) #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


### 2) Common Imports & Polite Headers

In [ ]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [ ]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [ ]:
# --- Q1 write your answer here ---
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")

import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
import io # Import io for using StringIO (good practice for literal strings)

url = "https://www.iban.com/country-codes"
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}

def fetch_html(url: str, timeout: int = 20) -> str:
    """Fetches HTML content from a given URL."""
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text

def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    """Flattens MultiIndex headers to a single string header."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df

print("Common helpers loaded.")
def q1_read_table(html: str) -> pd.DataFrame:
  tables = pd.read_html(io.StringIO(html))
  target_df = None
  for df in tables:
        if df.shape[1] >= 4:
            target_df = df
            break

  if target_df is None:
        raise ValueError("Could not find a table with at least 4 columns.")
  target_df = flatten_headers(target_df)
  return target_df

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
  col_map = {}
  for col in df.columns:
    col_lower = col.lower().strip().replace(' ', '')
    if 'country' in col_lower:
        col_map[col] = 'Country'
    elif 'alpha-2' in col_lower or 'alpha2' in col_lower:
         col_map[col] = 'Alpha-2'
    elif 'alpha-3' in col_lower or 'alpha3' in col_lower:
        col_map[col] = 'Alpha-3'
    elif 'numeric' in col_lower:
        col_map[col] = 'Numeric'

  required_cols = ['Country', 'Alpha-2', 'Alpha-3', 'Numeric']
  selected_cols = {old: new for old, new in col_map.items() if new in required_cols}
  df_clean = df[list(selected_cols.keys())].rename(columns=selected_cols)

  if not all(col in df_clean.columns for col in required_cols):
        missing = [c for c in required_cols if c not in df_clean.columns]
        raise ValueError(f"Missing required columns after mapping: {missing}")

  for col in ['Country', 'Alpha-2', 'Alpha-3']:
    df_clean[col] = df_clean[col].str.strip().str.upper()

  df_clean['Alpha-2'] = df_clean['Alpha-2'].str.upper()
  df_clean['Alpha-3'] = df_clean['Alpha-3'].str.upper()

  df_clean['Numeric'] = pd.to_numeric(
        df_clean['Numeric'].astype(str).str.strip(),
        errors='coerce'
    ).astype('Int64')

  df_clean.dropna(subset=['Alpha-2', 'Alpha-3', 'Numeric'], inplace=True)
  return df_clean

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    df = df.sort_values(by='Numeric', ascending=False)
    return df.head(top)

try:
    html_content = fetch_html(url)
    raw_df = q1_read_table(html_content)
    cleaned_df = q1_clean(raw_df.copy())
    csv_filename = "data_q1.csv"
    cleaned_df.to_csv(csv_filename, index=False)
    print(f"Data successfully written to {csv_filename}")

    top_items_df = q1_sort_top(cleaned_df, top=15)
    print("\n--- Top 15 IBAN Country Codes by Numeric (Descending) ---")
    print(top_items_df.to_string(index=False))

except requests.exceptions.HTTPError as e:
    print(f"Error fetching URL: HTTP {e.response.status}")


Dependencies installed.
Common helpers loaded.
Data successfully written to data_q1.csv

--- Top 15 IBAN Country Codes by Numeric (Descending) ---
                                                   Country Alpha-2 Alpha-3  Numeric
                                                    ZAMBIA      ZM     ZMB      894
                                                     YEMEN      YE     YEM      887
                                                     SAMOA      WS     WSM      882
                                         WALLIS AND FUTUNA      WF     WLF      876
                        VENEZUELA (BOLIVARIAN REPUBLIC OF)      VE     VEN      862
                                                UZBEKISTAN      UZ     UZB      860
                                                   URUGUAY      UY     URY      858
                                              BURKINA FASO      BF     BFA      854
                                     VIRGIN ISLANDS (U.S.)      VI     VIR      850
             

## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [3]:
# Q2 — Write your answer here
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")

import re
import pandas as pd
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

URL = "https://news.ycombinator.com/"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
    )
}

def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")

def q2_parse_items(html: str) -> pd.DataFrame:
    soup = BeautifulSoup(html, "lxml")
    items = []

    for row in soup.find_all("tr", class_="athing"):
        data = {}

        rank_tag = row.find("span", class_="rank")
        data["rank"] = rank_tag.get_text(strip=True) if rank_tag else None

        titleline = row.find("span", class_="titleline")
        a = titleline.find("a") if titleline else None
        data["title"] = a.get_text(strip=True) if a else None
        href = a.get("href") if a else None
        data["link"] = urljoin(URL, href) if href else None

        sub = row.find_next_sibling("tr")
        points_text = user_text = comments_text = None

        if sub:
            subtext_td = sub.find("td", class_="subtext")
            if subtext_td:
                score = subtext_td.find("span", class_="score")
                points_text = score.get_text(strip=True) if score else None

                user_tag = subtext_td.find("a", class_="hnuser")
                user_text = user_tag.get_text(strip=True) if user_tag else None

                comment_link = None
                for a_tag in subtext_td.find_all("a"):
                    txt = a_tag.get_text(strip=True).lower()
                    if "comment" in txt or txt == "discuss":
                        comment_link = a_tag
                comments_text = comment_link.get_text(strip=True) if comment_link else None

        data["points"] = points_text
        data["user"] = user_text
        data["comments"] = comments_text

        items.append(data)

    return pd.DataFrame(items)

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in ["rank", "points", "comments"]:
        df[col] = df[col].apply(safe_int_extract).astype(int)
    for col in ["title", "link", "user"]:
        df[col] = df[col].fillna("").astype(str).str.strip()
    return df

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    return df.sort_values(by="points", ascending=False).head(top).reset_index(drop=True)

try:
    html = fetch_html(URL)
    raw = q2_parse_items(html)
    cleaned = q2_clean(raw)
    csv_filename = "data_q2.csv"
    cleaned.to_csv(csv_filename, index=False)
    print(f"Data successfully written to {csv_filename}")

    top15 = q2_sort_top(cleaned, 15)
    print("\n--- Top 15 by points ---")
    print(top15[["rank", "title", "points", "comments", "user"]].to_string(index=False))

except requests.exceptions.HTTPError as e:
    print(f"Error fetching URL: HTTP {e.response.status_code}. Cannot continue.")
except Exception as e:
    print(f"An unexpected error occurred: {e}. Check your network connection or the site structure.")


Dependencies installed.
Common helpers loaded.
Data successfully written to data_q2.csv

--- Top 15 by points ---
 rank                                                                          title  points  comments           user
    2                                               Solarpunk is happening in Africa     597       301        JoiDegn
    9            New gel restores dental enamel and could revolutionise tooth repair     360       151   CGMthrowaway
   22               I was right about dishwasher pods and now I can prove it [video]     316       202  hnaccount_rng
   17                                           The shadows lurking in the equations     258        81         calebm
    4                                  Dillo, a multi-platform graphical web browser     245        91   nazgulsenpai
    5 ChatGPT terms disallow its use in providing legal and medical advice to others     236       222 randycupertino
   27 SPy: An interpreter and compiler for a fast statically